# Find similar strings with FuzzyWuzzy

There are different ways to make data dirty, and inconsistent data entry is one of them. Inconsistent values are even worse than duplicates, and sometimes difficult to detect.<br>
In this notebook, I apply [FuzzyWuzzy](https://github.com/seatgeek/fuzzywuzzy) package to find similar ramen brand names in a ramen review dataset.<br>
Data source: [The ramen rater Big list](https://www.theramenrater.com/resources-2/the-list/)

## Import

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

## Overview of the dataset

In [2]:
# Load in the excel file
ramen = pd.read_excel('The-Ramen-Rater-The-Big-List-1-3400-Current-As-Of-Jan-25-2020.xlsx',
                      engine='openpyxl')

# Display the first columns
ramen.head()

,Review #,Brand,Variety,Style,Country,Stars
0,3400,EDO Pack,Kumamoto Flavour Noodles,Cup,Hong Kong,1
1,3399,Pan Mee,Goreng Dried Chili Shrimp Flavour,Pack,Malaysia,5
2,3398,Paldo,King Lid Ramen Noodle Soup,Pack,South Korea,5
3,3397,Nissin Miojo,Cremoso Carno Com Chili,Pack,Brazil,2
4,3396,Samyang Foods,Cham Ramen Big Bowl,Bowl,South Korea,2.25


In [3]:
# Check data type and null values
ramen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Review #  3400 non-null   int64 
 1   Brand     3400 non-null   object
 2   Variety   3400 non-null   object
 3   Style     3400 non-null   object
 4   Country   3400 non-null   object
 5   Stars     3400 non-null   object
dtypes: int64(1), object(5)
memory usage: 159.5+ KB


In [4]:
# Remove leading and trailing spaces in each string value
for col in ramen[['Brand','Variety','Style','Country']]:
    ramen[col] = ramen[col].str.strip()
    print('Number of unique values in ' + str(col) +' is ' + str(ramen[col].nunique()))

Number of unique values in Brand is 499
Number of unique values in Variety is 3170
Number of unique values in Style is 8
Number of unique values in Country is 48


In [5]:
ramen['Country'].unique()

array(['Hong Kong', 'Malaysia', 'South Korea', 'Brazil', 'Japan',
       'United States', 'Taiwan', 'Mexico', 'China', 'Canada', 'Peru',
       'Spain', 'Thailand', 'Indonesia', 'Portugal', 'United Kingdom',
       'Italy', 'Russia', 'Singapore', 'Philippines', 'Bangladesh',
       'France', 'Australia', 'Vietnam', 'New Zealand', 'Ukraine',
       'Sarawak', 'Netherlands', 'Poland', 'Germany', 'USA', 'India',
       'Phlippines', 'Nigeria', 'Ghana', 'Hungary', 'Fiji', 'Pakistan',
       'Nepal', 'UK', 'Myanmar', 'Cambodia', 'Finland', 'Sweden',
       'Colombia', 'Estonia', 'Holland', 'Dubai'], dtype=object)

I would like to know if one brand can have multiple manufacturers in different countries.

In [6]:
brand_country = ramen['Brand'] +' '+ ramen['Country']
brand_country.nunique()

574

574 is greater than 499 brands, so yes, one brand can have different country values. Next, we need to get the list of unique brand names.

In [7]:
unique_brand = ramen['Brand'].unique().tolist()
sorted(unique_brand)[:20]

['1 To 3 Noodles',
 '7 Select',
 '7 Select/Nissin',
 '7-Eleven / Nissin',
 'A-One',
 'A-Sha',
 'A-Sha Dry Noodle',
 'A1',
 'ABC',
 'Acecook',
 'Adabi',
 'Ah Lai',
 'Aji-no-men',
 'Ajinatori',
 'Ajinomoto',
 'Alhami',
 'Amianda',
 'Amino',
 "Annie Chun's",
 'Aroi']

We can see some suspicious names right at the beginning. Let's have some tests first.

## FuzzyWuzzy

FuzzyWuzzy has four scorer options to find the Levenshtein distance between two strings. In this example, I would check on the token sort ratio and the token set ratio, for I believe they are more suitable for this dataset which might have mixed words order and duplicated words.<br>
I would pick four brand names and find their similar names in the Brand column. Since we're matching the Brand column with itself, the result would always include the selected name with a score of 100.

### Token Sort Ratio
The token sort ratio scorer tokenizes the strings and cleans them by returning these strings to lower cases, removing punctuations, and then sorting them alphabetically. After that, it finds the Levenshtein distance and returns the similarity percentage.

In [8]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_sort_ratio)

[('7 Select', 100),
 ('7 Select/Nissin', 70),
 ('Jinbo Selection', 61),
 ('Seven & I', 53),
 ('Lele', 50)]

This result means that '7 Select/Nissin' has 70% similarity when referring to '7 Select'. Not bad if I set the threshold at 70% to get the pair of 7 Select - 7 Select/Nissin.

In [9]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_sort_ratio)

[('A-Sha', 100), ('Shan', 67), ('Nasoya', 55), ('Alhami', 55), ('Ah Lai', 55)]

Below 70%, there's no match for A-sha.

In [10]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_sort_ratio)

[('Acecook', 100),
 ('Vina Acecook', 74),
 ('Yatekomo', 53),
 ('Sahmyook', 53),
 ('Panco', 50)]

Still good at 70% threshold.

In [11]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_sort_ratio)

[("Chef Nic's Noodles", 100),
 ("Mr. Lee's Noodles", 71),
 ('Fantastic Noodles', 69),
 ('1 To 3 Noodles', 62),
 ("Mom's Dry Noodle", 59)]

Now, we have a problem here. Token sort ratio scorer will gets the wrong pair of Chef Nic's Noodles - Mr. Lee's Noodles if I set 70% threshold.

In [12]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_sort_ratio)

[('Chorip Dong', 100),
 ('ChoripDong', 95),
 ('Hi-Myon', 56),
 ('Mr. Udon', 56),
 ('Maison de Coree', 54)]

This one looks good enough.

### Token Set Ratio
The token set ratio scorer also tokenizes the strings, and follows processing steps just like the token sort ratio. Then it collects common tokens between two strings and performs pairwise comparisons to find the similarity percentage.

In [13]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_set_ratio)

[('7 Select', 100),
 ('7 Select/Nissin', 100),
 ('The Ramen Rater Select', 86),
 ('Jinbo Selection', 61),
 ('Seven & I', 53)]

Since the token set ratio is more flexible, the score has increased from 70% to 100% for 7 Select - 7 Select/Nissin.

In [14]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_set_ratio)

[('A-Sha', 100),
 ('A-Sha Dry Noodle', 100),
 ('Shan', 67),
 ('Nasoya', 55),
 ('Alhami', 55)]

Now we see A-Sha has another name as A-Sha Dry Noodle. And we can see this only by using token set ratio.

In [15]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_set_ratio)

[('Acecook', 100),
 ('Vina Acecook', 100),
 ('Yatekomo', 53),
 ('Sahmyook', 53),
 ('Panco', 50)]

This one got 100% just like the 7 Select case.

In [16]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_set_ratio)

[("Chef Nic's Noodles", 100),
 ('S&S', 100),
 ('Mr. Noodles', 82),
 ("Mr. Lee's Noodles", 72),
 ('Tseng Noodles', 70)]

This one got much worse when token set ratio returns 100% for the pair of Chef Nic's Noodles - S&S.

In [17]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_set_ratio)

[('Chorip Dong', 100),
 ('ChoripDong', 95),
 ('Hi-Myon', 56),
 ('Mr. Udon', 56),
 ('Maison de Coree', 54)]

We have the same result for this one.

Although the token set ratio is more flexible and can detect more similar strings than the token sort ratio, it might also bring in more wrong matches.

## Apply FuzzyWuzzy in one column

### Token Sort Ratio

We need to create a dataframe with brand names, matched brands, and their scores.

In [18]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_sort_ratio)]

In [19]:
#Create dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])
similarity_sort.head()

,brand_sort,match_sort,score_sort
0,EDO Pack,EDO Pack,100
1,EDO Pack,Gau Do,57
2,EDO Pack,Pan Mee,53
3,EDO Pack,Plats Du Chef,48
4,EDO Pack,Vedan,46


Since we're looking for matched values from the same column, one value pair would have another same pair in a reversed order. For example, we will find one pair of EDO Pack - Gau Do, and another pair of Gau Do - EDO Pack. To eliminate one of them later, I need to find a representative value for each of two same pairs.

In [20]:
#Derive representative values
similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])
similarity_sort.head()

,brand_sort,match_sort,score_sort,sorted_brand_sort
0,EDO Pack,EDO Pack,100,EDO Pack
1,EDO Pack,Gau Do,57,EDO Pack
2,EDO Pack,Pan Mee,53,EDO Pack
3,EDO Pack,Plats Du Chef,48,EDO Pack
4,EDO Pack,Vedan,46,EDO Pack


Based on the tests above, I would care about those pairs which have at least 80% similarity. I also exclude those which match to themselves (brand value and match value are exactly the same), and those which are duplicated pairs.

In [21]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                                      (similarity_sort['brand_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]

In [22]:
#Drop the representative value column
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

Now, let's see the result.

In [23]:
#Group matches by brand names and scores
#pd.set_option('display.max_rows', None)
high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
brand_sort,score_sort,
Nissin Miojo,100,Nissin-Miojo
Lau Liu Tou,100,Lau Liu tou
World O Noodle,100,World O' Noodle
Unif / Tung-I,100,Unif Tung-I
TableMark,100,Tablemark
MyKuali,100,Mykuali
Mamee Shinsegae,100,Mamee-Shinsegae
Mamee / Shinsegae,100,"Mamee Shinsegae, Mamee-Shinsegae"
MAMA,100,Mama


From the score of 95 and above, everything looks good. In each pair, the two values might have typos, one missing character, or inconsistent format, but overal they obviously refer to each other. Below 95, it would be harder to tell. We can look at some examples by listing out data from each pair.

In [24]:
#Souper - Super - 91%
ramen[(ramen['Brand'] == 'Souper') | (ramen['Brand'] == 'Super')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars
2583,817,Souper,Bowl Ramen Shrimp,Bowl,Taiwan,3.75
2987,413,Souper,Alimentary Paste Chow Mein,Pack,Taiwan,4.25
523,2877,Super,Cup Instant Noodle Seafood Flavour,Cup,Malaysia,2.5
592,2808,Super,Super Cup Signature White Curry,Cup,Malaysia,0.25
1610,1790,Super,Kimchi,Cup,Malaysia,4.5
1612,1788,Super,Curry Flavour Noodles,Cup,Malaysia,3
1615,1785,Super,Instant Noodles Black Pepper Crab Mi Goreng,Cup,Malaysia,3.5
1668,1732,Super,Seafood Flavour Noodles,Cup,Malaysia,3.25
1905,1495,Super,Instant Noodles Mee Goreng,Cup,Malaysia,3.75


For this pair, we see that these two brands come from different manufacturers, and there's also no similarity in their ramen types or styles. I would say that these brands are not the same.

In [25]:
#Sura - Suraj - 89%
ramen[(ramen['Brand'] == 'Sura') | (ramen['Brand'] == 'Suraj')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars
2756,644,Sura,Knife Cut Rice Noodle,Bowl,South Korea,3.75
285,3115,Suraj,Masala Noodles,Pack,Canada,2


Sura and Suraj are two different brands.

In [26]:
#Ped Chef - Red Chef - 88%
ramen[(ramen['Brand'] == 'Ped Chef') | (ramen['Brand'] == 'Red Chef')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars
637,2763,Ped Chef,Sesame Clear Soup Noodles,Pack,Malaysia,3.5
164,3236,Red Chef,Spicy Sakura Prawn Soup Rice Vermicelli & Noodles,Pack,Malaysia,5
198,3202,Red Chef,Pandan White Curry Noodle (New Recipe),Pack,Malaysia,5
638,2762,Red Chef,Pandan White Curry Noodles,Pack,Malaysia,5
639,2761,Red Chef,Green Tom Yum Soup Noodles,Pack,Malaysia,5
640,2760,Red Chef,Spicy Sakura Prawn Soup Noodles,Pack,Malaysia,5


Here, we only have one record of the Ped Chef brand, and we also see the same pattern in its variety name in comparison with the Red Chef brand. I'm pretty sure these two brands are the same.

We can continue to check other pairs by the same method. From the threshold of 84% and below, we can ignore some pairs which are obviously different or we can make a quick check as above. Next, I will apply token set ratio scorer to find matched brand names, and then compare the results.

### Token Set Ratio

We will go over the same steps as above.

In [27]:
#Create tuples of brand names, matched brand names, and the score
score_set = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_set_ratio)]

In [28]:
#Create dataframe from the tuples and derive representative values
similarity_set = pd.DataFrame(score_set, columns=['brand_set','match_set','score_set'])
similarity_set['sorted_brand_set'] = np.minimum(similarity_set['brand_set'], similarity_set['match_set'])

#Pick values
high_score_set = similarity_set[(similarity_set['score_set'] >= 80) & 
                                    (similarity_set['brand_set'] != similarity_set['match_set']) & 
                                    (similarity_set['sorted_brand_set'] != similarity_set['match_set'])]

#Drop the representative value column
high_score_set = high_score_set.drop('sorted_brand_set',axis=1).copy()

Since the token set ratio scorer will tolerate more 'noise' when matching two values. I would group the result by matched values to reduce the number of rows.

In [29]:
#Group brands by matches and scores
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
high_score_set.groupby(['match_set','score_set']).agg(
                       {'brand_set': ', '.join}).sort_values(
                       ['score_set'], ascending=False)

,,brand_set
match_set,score_set,
7 Select/Nissin,100,7 Select
Tiger Tiger,100,Tiger
President Rice,100,President
Prima Taste,100,Prima
Q,100,Little Couples Q Noodle
S&S,100,"Chef Nic's Noodles, Mama Pat's, Mom's Dry Noodle, Culley's, Mike's Mighty Good Craft Ramen, Ching's Secret, Dr. McDougall's, Farmer's Heart, Mr. Lee's Noodles, Chaudhary's Wai Wai, Campbell's, Annie Chun's, Han's South Korea, Kim's Bowl"
Sakurai Foods,100,Sakurai
Sichuan Baijia,100,Baijia
Six Fortune,100,Fortune


## Comparison

In this part, I will create a merged table including results from token sort ratio and token set ratio with some changes.<br>
The tables will be merged by matched values to shorten the result table, and because I would like to keep the scores after grouping all values, I need to create new columns which combine brand names and scores. 

In [30]:
#Create columns with brand names combining scores
high_score_sort['brand_sort'] = high_score_sort['brand_sort'] + ': ' + high_score_sort['score_sort'].astype(str)
high_score_set['brand_set'] = high_score_set['brand_set'] + ': ' + high_score_set['score_set'].astype(str)

Now, I can group the two tables by matched names, and then rename the columns

In [31]:
#Group data by matched name and store in new dataframe
token_sort = high_score_sort.groupby(['match_sort']).agg({'brand_sort': ', '.join}).reset_index()
token_set = high_score_set.groupby(['match_set']).agg({'brand_set': ', '.join}).reset_index()

#Rename columns
token_sort = token_sort.rename(columns={'match_sort':'brand'})
token_set = token_set.rename(columns={'match_set':'brand'})

I would choose 'outer' option to get all the data from two tables.

In [32]:
#Outer join two tables by brand (matched names)
similarity = pd.merge(token_sort, token_set, how='outer', on='brand')

#Replace NaN values and rename columns for readability
similarity = similarity.replace(np.nan,'')
similarity = similarity.rename(columns={'brand_set':'token_set_ratio','brand_sort':'token_sort_ratio'})

In [33]:
similarity.sort_values('brand').reset_index(drop=True)

,brand,token_sort_ratio,token_set_ratio
0,7 Select/Nissin,,7 Select: 100
1,7-Eleven / Nissin,,7 Select/Nissin: 80
2,A-Sha Dry Noodle,,A-Sha: 100
3,ChoripDong,Chorip Dong: 95,Chorip Dong: 95
4,Fantastic Noodles,,Fantastic: 100
5,Fashion Foods,Fashion Food: 96,Fashion Food: 96
6,Goku-Uma,Goku Uma: 100,Goku Uma: 100
7,Hokkaido Ramen Santouka,,Hokkaido Furano Ramen: 82
8,Hua Feng Noodle Expert,,Hua Feng: 100
9,Ishimaru,Higashimaru: 84,Higashimaru: 84


Now we can see how different it is between two scorers. As expected, the token set ratio matches wrong names with high scores (e.g. S&S, Mr.Noodles). However, it does bring in more matches that the token sort ratio could not get (e.g. 7 Select/Nissin, Sugakiya Foods, Vina Acecook).<br>
It would be beneficial to apply both methods in this case.